# Baixando dados realizados pela Lei de Acesso à Informação (LAI) de Pedidos e Recursos/Reclamações.

Nesse notebook vamos apresentar como podemos, utilizando a linguagem de programação python, baixar de forma mais automática dos dados da lai de pedidos e recursos/reclamações. 

------------------------------
-- CAMPOS: PEDIDOS
------------------------------
- IdPedido - inteiro: identificador único do pedido (não mostrado no sistema);
- ProtocoloPedido - texto(17): número do protocolo do pedido;
- Esfera - texto(30): esfera do órgão destinatário do pedido;
- UF - texto(2): Sigla da UF do pedido quando não federal; 
- Municipio - texto(200): Nome do município do pedido quando não federal;
- OrgaoDestinatario  - texto(250): nome do órgão destinatário do pedido;
- Situacao - texto(200): descrição da situação do pedido;
- DataRegistro - Data DD/MM/AAAA HH:MM:SS : data de abertura do pedido;
- PrazoAtendimento - Data DD/MM/AAAA HH:MM:ss : data limite para atendimento ao pedido;
- FoiProrrogado - texto(3) "Sim" ou "Não" : informa se houve prorrogação do prazo do pedido;
- FoiReencaminhado - texto(3) "Sim" ou "Não": informa se o pedido foi reencaminhado;
- FormaResposta - texto(200): tipo de resposta escolhida pelo solicitante na abertura do pedido;
- OrigemSolicitacao - texto(50): informa se o pedido foi aberto em um Balcão SIC ou pela Internet;
- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- AssuntoPedido - texto(200) : assunto do pedido atribuído pel SIC;
- SubAssuntoPedido - texto(200) : subassunto do pedido atribuída pelo SIC;
- Tag - texto(1024): as tags são marcadores no pedido para realizar classificações que não estão em assuntos/ subassuntos;
- DataResposta - Data DD/MM/AAAA HH:MM:SS : data da resposta ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");
- Decisao - texto(100) : tipo resposta dada ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");
- EspecificacaoDecisao - texto(200): subtipo da resposta dada ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");

------------------------------
-- CAMPOS: RECURSOS
------------------------------

- IdRecurso - inteiro: identificador único do recurso (não mostrado no sistema);
- IdRecursoPrecedente - inteiro: identificador único do recurso que precedeu este (não mostrado no sistema e em branco no caso de Recursos de 1ª Instância e Reclamações);
- IdPedido - inteiro: identificador único do pedido ao qual o recurso pertence (não mostrado no sistema);
- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- ProtocoloPedido - texto(17): número do protocolo do pedido ao qual o recurso pertence;
- Esfera - texto(30): descrição da esfera do recurso;
- UF - texto(2): Sigla da UF do recurso quando não federal;
- Municipio - texto(200): Nome do município do recurso quando não federal;
- OrgaoDestinatario - texto(250): nome do órgão destinatário do recurso;
- Instancia - texto(80): descrição da instância do recurso;
- Situacao - texto(80): descrição da situação do recurso;
- DataRegistro - Data DD/MM/AAAA HH:MM:SS : data de abertura do recurso;
- PrazoAtendimento - Data DD/MM/AAAA HH:MM:SS : data limite para atendimento ao recurso;
- OrigemSolicitacao - texto(50): informa se o recurso foi aberto em um Balcão SIC ou pela Internet;
- TipoRecurso - texto(80): motivo de abertura do recurso;
- DataResposta - Data DD/MM/AAAA HH:MM:SS : data da resposta ao recurso (campo em branco para recursos que ainda estejam na situação "Em Tramitação");
- TipoResposta - texto(80): tipo resposta dada ao recurso (campo em branco para recursos que ainda estejam na situação "Em Tramitação");

------------------------------
-- CAMPOS: SOLICITANTES
------------------------------

- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- TipoDemandante - texto(15): Pessoa Fìsica ou Pessoa Jurídica;
- DataNascimento - Data DD/MM/AAAA : data de nascimento do solicitante;
- Genero - texto(13) : Masculino, Feminino ou Outro(em branco para pessoa jurídica);
- Escolaridade - texto(200): Escolaridade do solicitante (em branco para pessoa jurídica);
- Profissao - texto(200): Profissão do solicitante (em branco para pessoa jurídica);
- TipoPessoaJuridica - texto(200): tipo de Pessoa Jurídica do solicitante (em branco para pessoa física)
- Pais - texto(200): país de residência do solicitante;
- UF - texto(2): UF de residência do solicitante;
- Municipio - texto(200): Município de residência do solicitante;

In [1]:
import requests
from datetime import date
import shutil
import os
import time
import zipfile
import pandas as pd
import re


In [2]:
header_pedidos = ['IdPedido',
                'ProtocoloPedido',
                'Esfera',
                'UF',
                'Municipio',
                'OrgaoDestinatario',
                'Situacao',
                'DataRegistro',
                'PrazoAtendimento',
                'FoiProrrogado',
                'FoiReencaminhado',
                'FormaResposta',
                'OrigemSolicitacao',
                'IdSolicitante',
                'AssuntoPedido',
                'SubAssuntoPedido',
                'Tag',
                'DataResposta',
                'Decisao',
                'EspecificacaoDecisao']

header_recursos = ['IdRecurso',
                'IdRecursoPrecedente',
                'IdPedido',
                'IdSolicitante',
                'ProtocoloPedido',
                'Esfera',
                'UF',
                'Municipio',
                'OrgaoDestinatario',
                'Instancia',
                'Situacao',
                'DataRegistro',
                'PrazoAtendimento',
                'OrigemSolicitacao',
                'TipoRecurso',
                'DataResposta',
                'TipoResposta']

header_solicitante = ['IdSolicitante',
                    'TipoDemandante',
                    'DataNascimento',
                    'Genero',
                    'Escolaridade',
                    'Profissao',
                    'TipoPessoaJuridica',
                    'Pais',
                    'UF',
                    'Municipio']

In [22]:
ANO_INICIO=2012
PREFIXO_ACESSO = "Pedidos"
PREFIXO_RECURSO = "Recursos_Reclamacoes"
NOME_ARQUIVO_BASE = "{prefixo}_{formato_arquivo}_{ano}.zip"
URL_BASE = "https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/{nome_arquivo}"

NOME_ARQUIVO_FILTRADO = 'Arquivos_{formato_arquivo}_{ano}.zip'
URL_FILTRADOS = "https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/{nome_arquivo}"


In [39]:
def busca_lai_por_anos(anos, tipo_dados='acesso', localizacao='./', formato_arquivo='csv'):
    """
    anos: Lista de anos para download.
    tipo_dados: Se o arquivo é de acesso ou recurso
    localizacao: Onde salvar os arquivos
    formato_arquivo: formatos xml|csv
    """
    if tipo_dados.lower() == "acesso":
        prefix = PREFIXO_ACESSO
    else:
        prefix = PREFIXO_RECURSO

    for year in anos:
        nome_arquivo  = NOME_ARQUIVO_BASE.format(prefixo=prefix, formato_arquivo=formato_arquivo, ano=str(year))
        url = URL_BASE.format(nome_arquivo=nome_arquivo)
        try:
            response = requests.get(url, stream=True)
            with open(os.path.join(localizacao, nome_arquivo), 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)

            print(f"Arquivo {nome_arquivo} baixado de {url}")
        except Exception as e:
            print(e)
            print(f"Error baixando arquivo {nome_arquivo} da url {url}")

        time.sleep(2)
        
def unzip_arquivos(de='zips', diretorio_destino='csvs'):
    #https://stackoverflow.com/questions/3451111/unzipping-files-in-python

    pattern = re.compile(".*\.zip")
    files = os.listdir(de)
    for file in files:
        if os.path.isfile(os.path.join(de, file)) and pattern.match(file.lower()) != None:    
            arquivo = os.path.join(de, file)
            with zipfile.ZipFile(arquivo,"r") as arquivo_zip:
                print(f'descompactando {arquivo}')
                arquivo_zip.extractall(diretorio_destino)
        

def busca_textos_por_anos(anos, localizacao='./', formato_arquivo='csv'):
    """
    anos: Lista de anos para download.
    localizacao: Onde salvar os arquivos
    formato_arquivo: formatos xml|csv
    """
    
    for year in anos:
        nome_arquivo  = NOME_ARQUIVO_FILTRADO.format(formato_arquivo=formato_arquivo, ano=str(year))
        url = URL_FILTRADOS.format(nome_arquivo=nome_arquivo)
        try:
            response = requests.get(url, stream=True)
            with open(os.path.join(localizacao, nome_arquivo), 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)

            print(f"Arquivo {nome_arquivo} baixado de {url}")
        except Exception as e:
            print(e)
            print(f"Error baixando arquivo {nome_arquivo} da url {url}")

        time.sleep(2)
        
        
class Pedidos:
    
    @staticmethod
    def carrega_arquivos_csv_df(diretorio, header_pedidos, parse_dates=None):
        pattern = re.compile(".*\_pedidos\_.*\.csv")
        df = pd.DataFrame()
        files = os.listdir(diretorio)
        if parse_dates is None:
            parse_dates=['DataRegistro', 'PrazoAtendimento', 'DataResposta']
            
        for file in files:
            if os.path.isfile(os.path.join(diretorio, file)) and pattern.match(file.lower()) != None:
                print(f"Carregando {file}")
                df = pd.concat([df, pd.read_csv(os.path.join(diretorio, file), names=header_pedidos, sep=';', encoding='utf-16', parse_dates=parse_dates, dayfirst=True, engine='c')])
                print(f'Carregado, memória utilizada após carga: {round(df.memory_usage(deep=True).sum()/(1024*1024), 2)}MB')
        return df

    @staticmethod
    def carrega_arquivos_xml_df(diretorio, header_pedidos, parse_dates):
        pattern = re.compile(".*\_pedidos\_.*\.xml")
        df = pd.DataFrame()
        files = os.listdir(diretorio)
        for file in files:
            if os.path.isfile(os.path.join(diretorio, file)) and pattern.match(file.lower()) != None:
                print(f"Carregando {file}")
                df = pd.concat([df, pd.read_xml(os.path.join(diretorio, file), names=header_pedidos, encoding='utf-16', parse_dates=parse_dates, dayfirst=True, engine='c')])
                print(f'Carregado, memória utilizada após carga: {round(df.memory_usage(deep=True).sum()/(1024*1024), 2)}MB')
        return df



In [5]:
anos = range(ANO_INICIO, int(date.today().strftime("%Y"))+1)
list(anos)


[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [24]:
busca_lai_por_anos(anos, 'acesso', 'zips_lai_rel', 'csv')


Arquivo Pedidos_csv_2015.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2015.zip
Arquivo Pedidos_csv_2016.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2016.zip
Arquivo Pedidos_csv_2017.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2017.zip
Arquivo Pedidos_csv_2018.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2018.zip
Arquivo Pedidos_csv_2019.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2019.zip
Arquivo Pedidos_csv_2020.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2020.zip
Arquivo Pedidos_csv_2021.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2021.zip
Arquivo Pedidos_csv_2022.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2022.zip


In [7]:
busca_lai_por_anos(anos, 'recurso', 'zips_lai_rel', 'csv')


Arquivo Recursos_Reclamacoes_csv_2012.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Recursos_Reclamacoes_csv_2012.zip
Arquivo Recursos_Reclamacoes_csv_2013.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Recursos_Reclamacoes_csv_2013.zip
Arquivo Recursos_Reclamacoes_csv_2014.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Recursos_Reclamacoes_csv_2014.zip
Arquivo Recursos_Reclamacoes_csv_2015.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Recursos_Reclamacoes_csv_2015.zip
Arquivo Recursos_Reclamacoes_csv_2016.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Recursos_Reclamacoes_csv_2016.zip
Arquivo Recursos_Reclamacoes_csv_2017.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Recursos_Reclamacoes_csv_2017.zip
Arquivo Recursos_Reclamacoes_csv_2018.zip baixado de https://dadosabertos-download.cgu.gov.br/

In [27]:
unzip_arquivos(de='zips_lai_rel', diretorio_destino='csvs_rel')
               

descompactando zips/Pedidos_csv_2023.zip
descompactando zips/Arquivos_csv_2018.zip
descompactando zips/Recursos_Reclamacoes_csv_2016.zip
descompactando zips/Recursos_Reclamacoes_csv_2015.zip
descompactando zips/Pedidos_csv_2021.zip
descompactando zips/Recursos_Reclamacoes_csv_2013.zip
descompactando zips/Pedidos_csv_2016.zip
descompactando zips/Arquivos_csv_2023.zip
descompactando zips/Pedidos_csv_2015.zip
descompactando zips/Recursos_Reclamacoes_csv_2014.zip
descompactando zips/Recursos_Reclamacoes_csv_2017.zip
descompactando zips/Pedidos_csv_2020.zip
descompactando zips/Arquivos_csv_2021.zip
descompactando zips/Pedidos_csv_2019.zip
descompactando zips/Arquivos_csv_2016.zip
descompactando zips/Recursos_Reclamacoes_csv_2020.zip
descompactando zips/Recursos_Reclamacoes_csv_2021.zip
descompactando zips/Pedidos_csv_2013.zip
descompactando zips/Arquivos_csv_2020.zip
descompactando zips/Pedidos_csv_2012.zip
descompactando zips/Pedidos_csv_2014.zip
descompactando zips/Recursos_Reclamacoes_cs

In [10]:
%%time
df = Pedidos.carrega_arquivos_csv_df('csvs_rel')


Carregando 20230227_Pedidos_csv_2015.csv
Carregado, memória utilizada após carga: 122.56MB
Carregando 20230227_Pedidos_csv_2021.csv
Carregado, memória utilizada após carga: 272.4MB
Carregando 20230227_Pedidos_csv_2014.csv
Carregado, memória utilizada após carga: 381.22MB
Carregando 20230227_Pedidos_csv_2012.csv
Carregado, memória utilizada após carga: 451.91MB
Carregando 20230227_Pedidos_csv_2018.csv
Carregado, memória utilizada após carga: 611.91MB
Carregando 20230227_Pedidos_csv_2016.csv
Carregado, memória utilizada após carga: 751.54MB
Carregando 20230227_Pedidos_csv_2023.csv
Carregado, memória utilizada após carga: 776.56MB
Carregando 20230227_Pedidos_csv_2013.csv
Carregado, memória utilizada após carga: 886.29MB
Carregando 20230222_Pedidos_csv_2023.csv
Carregado, memória utilizada após carga: 909.93MB
Carregando 20230222_Pedidos_csv_2016.csv
Carregado, memória utilizada após carga: 1049.56MB
Carregando 20230222_Pedidos_csv_2019.csv
Carregado, memória utilizada após carga: 1211.54M

In [11]:
df.head()

,IdPedido,ProtocoloPedido,Esfera,UF,Municipio,OrgaoDestinatario,Situacao,DataRegistro,PrazoAtendimento,FoiProrrogado,FoiReencaminhado,FormaResposta,OrigemSolicitacao,IdSolicitante,AssuntoPedido,SubAssuntoPedido,Tag,DataResposta,Decisao,EspecificacaoDecisao
0,1887813,99902001934201557,Federal,,,CEF – Caixa Econômica Federal,Concluída,2015-07-01,2015-07-21,Não,Não,Pelo sistema (com avisos por email),Internet,2547949,Outros em Economia e Finanças,,,15/07/2015,Acesso Concedido,Resposta solicitada inserida no Fala.Br
1,1887819,99902001935201500,Federal,,,CEF – Caixa Econômica Federal,Concluída,2015-07-01,2015-07-21,Não,Não,Pelo sistema (com avisos por email),Internet,2547949,Outros em Economia e Finanças,,,02/07/2015,Acesso Negado,Informação sigilosa classificada conforme a Le...
2,1887823,99902001936201546,Federal,,,CEF – Caixa Econômica Federal,Concluída,2015-07-01,2015-07-21,Não,Não,Pelo sistema (com avisos por email),Internet,2547949,Outros em Economia e Finanças,,,15/07/2015,Acesso Concedido,Resposta solicitada inserida no Fala.Br
3,1887829,46800000969201551,Federal,,,ME - Ministério da Economia,Concluída,2015-07-01,2015-07-21,Não,Não,Pelo sistema (com avisos por email),Internet,2565600,Proteção e Benefícios ao Trabalhador,,,09/07/2015,Acesso Concedido,Resposta solicitada inserida no Fala.Br
4,1887837,23480010257201512,Federal,,,UFPel – Fundação Universidade Federal de Pelotas,Concluída,2015-07-01,2015-07-21,Não,Não,Pelo sistema (com avisos por email),Internet,2564814,Conduta Docente,,,29/10/2015,Acesso Concedido,Resposta solicitada inserida no Fala.Br


## Arquivos com textos de pedidos

------------------------------
-- CAMPOS: PEDIDOS
------------------------------
- IdPedido - inteiro: identificador único do pedido (não mostrado no sistema);
- ProtocoloPedido - texto(17): número do protocolo do pedido;
- Esfera - texto(30): esfera do órgão destinatário do pedido; 
- OrgaoDestinatario  - texto(250): nome do órgão destinatário do pedido;
- Situacao - texto(200): descrição da situação do pedido;
- DataRegistro - Data DD/MM/AAAA HH:MM:SS : data de abertura do pedido;
- ResumoSolicitacao - texto(255): resumo do pedido;
- DetalhamentoSolicitacao - texto(2048): detalhamento do pedido;
- PrazoAtendimento - Data DD/MM/AAAA HH:MM:ss : data limite para atendimento ao pedido;
- FoiProrrogado - texto(3) "Sim" ou "Não" : informa se houve prorrogação do prazo do pedido;
- FoiReencaminhado - texto(3) "Sim" ou "Não": informa se o pedido foi reencaminhado;
- FormaResposta - texto(200): tipo de resposta escolhida pelo solicitante na abertura do pedido;
- OrigemSolicitacao - texto(50): informa se o pedido foi aberto em um Balcão SIC ou pela Internet;
- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- AssuntoPedido - texto(200) : assunto do pedido atribuído pel SIC;
- SubAssuntoPedido - texto(200) : subassunto do pedido atribuída pelo SIC;
- Tag - texto(1024): as tags são marcadores no pedido para realizar classificações que não estão em assuntos/ subassuntos;
- DataResposta - Data DD/MM/AAAA HH:MM:SS : data da resposta ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");
- Resposta - texto(8000): resposta ao pedido;
- Decisao - texto(100) : tipo resposta dada ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");
- EspecificacaoDecisao - texto(200): subtipo da resposta dada ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");


------------------------------
-- CAMPOS: RECURSOS
------------------------------

- IdRecurso - inteiro: identificador único do recurso (não mostrado no sistema);
- IdRecursoPrecedente - inteiro: identificador único do recurso que precedeu este (não mostrado no sistema e em branco no caso de Recursos de 1ª Instância e Reclamações);
- DescRecurso - texto(8000): descrição do recurso;
- IdPedido - inteiro: identificador único do pedido ao qual o recurso pertence (não mostrado no sistema);
- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- ProtocoloPedido - texto(17): número do protocolo do pedido ao qual o recurso pertence;
- OrgaoDestinatario - texto(250): nome do órgão destinatário do recurso;
- Instancia - texto(80): descrição da instância do recurso;
- Situacao - texto(80): descrição da situação do recurso;
- DataRegistro - Data DD/MM/AAAA HH:MM:SS : data de abertura do recurso;
- PrazoAtendimento - Data DD/MM/AAAA HH:MM:SS : data limite para atendimento ao recurso;
- OrigemSolicitacao - texto(50): informa se o recurso foi aberto em um Balcão SIC ou pela Internet;
- TipoRecurso - texto(80): motivo de abertura do recurso;
- DataResposta - Data DD/MM/AAAA HH:MM:SS : data da resposta ao recurso (campo em branco para recursos que ainda estejam na situação "Em Tramitação");
- RespostaRecurso - texto(8000): resposta ao recurso;
- TipoResposta - texto(80): tipo resposta dada ao recurso (campo em branco para recursos que ainda estejam na situação "Em Tramitação");

 
------------------------------
-- CAMPOS: SOLICITANTES
------------------------------

- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- TipoDemandante - texto(15): Pessoa Fìsica ou Pessoa Jurídica;
- DataNascimento - Data DD/MM/AAAA : data de nascimento do solicitante;
- Genero - texto(13) : Masculino, Feminino ou Outro(em branco para pessoa jurídica);
- Escolaridade - texto(200): Escolaridade do solicitante (em branco para pessoa jurídica);
- Profissao - texto(200): Profissão do solicitante (em branco para pessoa jurídica);
- TipoPessoaJuridica - texto(200): tipo de Pessoa Jurídica do solicitante (em branco para pessoa física)
- Pais - texto(200): país de residência do solicitante;
- UF - texto(2): UF de residência do solicitante;
- Municipio - texto(200): Município de residência do solicitante;

In [ ]:
pedidos_header_txt =[
- IdPedido - inteiro: identificador único do pedido (não mostrado no sistema);
- ProtocoloPedido - texto(17): número do protocolo do pedido;
- Esfera - texto(30): esfera do órgão destinatário do pedido; 
- OrgaoDestinatario  - texto(250): nome do órgão destinatário do pedido;
- Situacao - texto(200): descrição da situação do pedido;
- DataRegistro - Data DD/MM/AAAA HH:MM:SS : data de abertura do pedido;
- ResumoSolicitacao - texto(255): resumo do pedido;
- DetalhamentoSolicitacao - texto(2048): detalhamento do pedido;
- PrazoAtendimento - Data DD/MM/AAAA HH:MM:ss : data limite para atendimento ao pedido;
- FoiProrrogado - texto(3) "Sim" ou "Não" : informa se houve prorrogação do prazo do pedido;
- FoiReencaminhado - texto(3) "Sim" ou "Não": informa se o pedido foi reencaminhado;
- FormaResposta - texto(200): tipo de resposta escolhida pelo solicitante na abertura do pedido;
- OrigemSolicitacao - texto(50): informa se o pedido foi aberto em um Balcão SIC ou pela Internet;
- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- AssuntoPedido - texto(200) : assunto do pedido atribuído pel SIC;
- SubAssuntoPedido - texto(200) : subassunto do pedido atribuída pelo SIC;
- Tag - texto(1024): as tags são marcadores no pedido para realizar classificações que não estão em assuntos/ subassuntos;
- DataResposta - Data DD/MM/AAAA HH:MM:SS : data da resposta ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");
- Resposta - texto(8000): resposta ao pedido;
- Decisao - texto(100) : tipo resposta dada ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");
- EspecificacaoDecisao - texto(200): subtipo da resposta dada ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");


------------------------------
-- CAMPOS: RECURSOS
------------------------------

- IdRecurso - inteiro: identificador único do recurso (não mostrado no sistema);
- IdRecursoPrecedente - inteiro: identificador único do recurso que precedeu este (não mostrado no sistema e em branco no caso de Recursos de 1ª Instância e Reclamações);
- DescRecurso - texto(8000): descrição do recurso;
- IdPedido - inteiro: identificador único do pedido ao qual o recurso pertence (não mostrado no sistema);
- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- ProtocoloPedido - texto(17): número do protocolo do pedido ao qual o recurso pertence;
- OrgaoDestinatario - texto(250): nome do órgão destinatário do recurso;
- Instancia - texto(80): descrição da instância do recurso;
- Situacao - texto(80): descrição da situação do recurso;
- DataRegistro - Data DD/MM/AAAA HH:MM:SS : data de abertura do recurso;
- PrazoAtendimento - Data DD/MM/AAAA HH:MM:SS : data limite para atendimento ao recurso;
- OrigemSolicitacao - texto(50): informa se o recurso foi aberto em um Balcão SIC ou pela Internet;
- TipoRecurso - texto(80): motivo de abertura do recurso;
- DataResposta - Data DD/MM/AAAA HH:MM:SS : data da resposta ao recurso (campo em branco para recursos que ainda estejam na situação "Em Tramitação");
- RespostaRecurso - texto(8000): resposta ao recurso;
- TipoResposta - texto(80): tipo resposta dada ao recurso (campo em branco para recursos que ainda estejam na situação "Em Tramitação");

 
------------------------------
-- CAMPOS: SOLICITANTES
------------------------------

- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- TipoDemandante - texto(15): Pessoa Fìsica ou Pessoa Jurídica;
- DataNascimento - Data DD/MM/AAAA : data de nascimento do solicitante;
- Genero - texto(13) : Masculino, Feminino ou Outro(em branco para pessoa jurídica);
- Escolaridade - texto(200): Escolaridade do solicitante (em branco para pessoa jurídica);
- Profissao - texto(200): Profissão do solicitante (em branco para pessoa jurídica);
- TipoPessoaJuridica - texto(200): tipo de Pessoa Jurídica do solicitante (em branco para pessoa física)
- Pais - texto(200): país de residência do solicitante;
- UF - texto(2): UF de residência do solicitante;
- Municipio - texto(200): Município de residência do solicitante;

In [36]:
pedido_header_txts = ['IdPedido',
                'ProtocoloPedido',
                'Esfera',
                'OrgaoDestinatario',
                'Situacao',
                'DataRegistro',
                'ResumoSolicitacao',
                'DetalhamentoSolicitacao',
                'PrazoAtendimento',
                'FoiProrrogado',
                'FoiReencaminhado',
                'FormaResposta',
                'OrigemSolicitacao',
                'IdSolicitante',
                'AssuntoPedido',
                'SubAssuntoPedido',
                'Tag',
                'DataResposta',
                'Resposta',
                'Decisao',
                'EspecificacaoDecisao']

recurso_header_txts = ['IdRecurso',
                'IdRecursoPrecedente',
                'DescRecurso',
                'IdPedido',
                'IdSolicitante',
                'ProtocoloPedido',
                'OrgaoDestinatario',
                'Instancia',
                'Situacao',
                'DataRegistro',
                'PrazoAtendimento',
                'OrigemSolicitacao',
                'TipoRecurso',
                'DataResposta',
                'RespostaRecurso',
                'TipoResposta']


solicitante_header_txts = ['IdSolicitante',
                'TipoDemandante',
                'DataNascimento',
                'Genero',
                'Escolaridade',
                'Profissao',
                'TipoPessoaJuridica',
                'Pais',
                'UF',
                'Municipio']


In [19]:
anos = range(2015, int(date.today().strftime("%Y"))+1)
list(anos)


[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [25]:
busca_textos_por_anos(anos, localizacao='zips_lai_txts', formato_arquivo='csv')


https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2015.zip
Arquivo Arquivos_csv_2015.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2015.zip
https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2016.zip
Arquivo Arquivos_csv_2016.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2016.zip
https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2017.zip
Arquivo Arquivos_csv_2017.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2017.zip
https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2018.zip
Arquivo Arquivos_csv_2018.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2018.zip
https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arq

In [28]:
unzip_arquivos(de='zips_lai_txts', diretorio_destino='csvs_txts')


descompactando zips_lai_txts/Arquivos_csv_2018.zip
descompactando zips_lai_txts/Arquivos_csv_2023.zip
descompactando zips_lai_txts/Arquivos_csv_2021.zip
descompactando zips_lai_txts/Arquivos_csv_2016.zip
descompactando zips_lai_txts/Arquivos_csv_2020.zip
descompactando zips_lai_txts/Arquivos_csv_2019.zip
descompactando zips_lai_txts/Arquivos_csv_2022.zip
descompactando zips_lai_txts/Arquivos_csv_2015.zip
descompactando zips_lai_txts/Arquivos_csv_2017.zip


In [40]:
%%time
df = Pedidos.carrega_arquivos_csv_df('csvs_txts', header_pedidos=pedido_header_txts)

Carregando 20230227_Pedidos_csv_2015.csv
Carregado, memória utilizada após carga: 219.54MB
Carregando 20230227_Pedidos_csv_2021.csv
Carregado, memória utilizada após carga: 521.36MB
Carregando 20230227_Pedidos_csv_2018.csv
Carregado, memória utilizada após carga: 808.18MB
Carregando 20230227_Pedidos_csv_2016.csv
Carregado, memória utilizada após carga: 1061.32MB
Carregando 20230227_Pedidos_csv_2023.csv
Carregado, memória utilizada após carga: 1094.77MB
Carregando 20230227_Pedidos_csv_2022.csv
Carregado, memória utilizada após carga: 1351.24MB
Carregando 20230227_Pedidos_csv_2020.csv
Carregado, memória utilizada após carga: 1772.77MB
Carregando 20230227_Pedidos_csv_2019.csv
Carregado, memória utilizada após carga: 2088.37MB
Carregando 20230227_Pedidos_csv_2017.csv
Carregado, memória utilizada após carga: 2355.35MB
CPU times: user 18.1 s, sys: 1.81 s, total: 19.9 s
Wall time: 19.9 s


In [41]:
df.head()

,IdPedido,ProtocoloPedido,Esfera,OrgaoDestinatario,Situacao,DataRegistro,ResumoSolicitacao,DetalhamentoSolicitacao,PrazoAtendimento,FoiProrrogado,...,FormaResposta,OrigemSolicitacao,IdSolicitante,AssuntoPedido,SubAssuntoPedido,Tag,DataResposta,Resposta,Decisao,EspecificacaoDecisao
0,1887837,23480010257201512,Federal,UFPel – Fundação Universidade Federal de Pelotas,Concluída,2015-07-01,Aproveitamento,"Prezados, Gostaria de solicitar informações s...",2015-07-21,Não,...,Pelo sistema (com avisos por email),Internet,2564814,Conduta Docente,,,2015-10-29,"Boa tarde, Em primeiro lugar, pedimos desculpa...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
1,1887842,23480010258201559,Federal,UFRGS – Universidade Federal do Rio Grande do Sul,Concluída,2015-07-01,Aproveitamento,"Prezados, Gostaria de solicitar informações s...",2015-07-21,Não,...,Pelo sistema (com avisos por email),Internet,2564814,Outros em Trabalho,,,2015-07-02,Senhor Jomar Em atendimento à sua solicitação...,Acesso Concedido,Resposta solicitada inserida no Fala.Br
2,1887846,23480010259201501,Federal,"IFPI – Instituto Federal de Educação, Ciência ...",Concluída,2015-07-01,Aproveitamento,"Prezados, Gostaria de solicitar informações s...",2015-07-21,Não,...,Pelo sistema (com avisos por email),Internet,2564814,Conduta Docente,,,2016-11-29,Não há vagas disponíveis para o referido cargo.,Acesso Concedido,Resposta solicitada inserida no Fala.Br
3,1887851,23480010260201528,Federal,UFCG – Universidade Federal de Campina Grande,Concluída,2015-07-01,Aproveitamento,"Prezados, Gostaria de solicitar informações s...",2015-07-21,Não,...,Pelo sistema (com avisos por email),Internet,2564814,Educação Superior,,,2015-07-01,"Não dispomos de vaga em aberto, para o cargo d...",Acesso Concedido,Informações enviadas por e-mail
4,1887855,23480010261201572,Federal,UFF – Universidade Federal Fluminense,Concluída,2015-07-01,Aproveitamento,"Prezados, Gostaria de solicitar informações s...",2015-07-21,Não,...,Pelo sistema (com avisos por email),Internet,2564814,Educação Superior,,,2015-07-08,"Prezado, Segue resposta formulada por Aline d...",Acesso Concedido,Resposta solicitada inserida no Fala.Br


In [42]:
df.head()

,IdPedido,ProtocoloPedido,Esfera,OrgaoDestinatario,Situacao,DataRegistro,ResumoSolicitacao,DetalhamentoSolicitacao,PrazoAtendimento,FoiProrrogado,...,FormaResposta,OrigemSolicitacao,IdSolicitante,AssuntoPedido,SubAssuntoPedido,Tag,DataResposta,Resposta,Decisao,EspecificacaoDecisao
0,1887837,23480010257201512,Federal,UFPel – Fundação Universidade Federal de Pelotas,Concluída,2015-07-01,Aproveitamento,"Prezados, Gostaria de solicitar informações s...",2015-07-21,Não,...,Pelo sistema (com avisos por email),Internet,2564814,Conduta Docente,,,2015-10-29,"Boa tarde, Em primeiro lugar, pedimos desculpa...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
1,1887842,23480010258201559,Federal,UFRGS – Universidade Federal do Rio Grande do Sul,Concluída,2015-07-01,Aproveitamento,"Prezados, Gostaria de solicitar informações s...",2015-07-21,Não,...,Pelo sistema (com avisos por email),Internet,2564814,Outros em Trabalho,,,2015-07-02,Senhor Jomar Em atendimento à sua solicitação...,Acesso Concedido,Resposta solicitada inserida no Fala.Br
2,1887846,23480010259201501,Federal,"IFPI – Instituto Federal de Educação, Ciência ...",Concluída,2015-07-01,Aproveitamento,"Prezados, Gostaria de solicitar informações s...",2015-07-21,Não,...,Pelo sistema (com avisos por email),Internet,2564814,Conduta Docente,,,2016-11-29,Não há vagas disponíveis para o referido cargo.,Acesso Concedido,Resposta solicitada inserida no Fala.Br
3,1887851,23480010260201528,Federal,UFCG – Universidade Federal de Campina Grande,Concluída,2015-07-01,Aproveitamento,"Prezados, Gostaria de solicitar informações s...",2015-07-21,Não,...,Pelo sistema (com avisos por email),Internet,2564814,Educação Superior,,,2015-07-01,"Não dispomos de vaga em aberto, para o cargo d...",Acesso Concedido,Informações enviadas por e-mail
4,1887855,23480010261201572,Federal,UFF – Universidade Federal Fluminense,Concluída,2015-07-01,Aproveitamento,"Prezados, Gostaria de solicitar informações s...",2015-07-21,Não,...,Pelo sistema (com avisos por email),Internet,2564814,Educação Superior,,,2015-07-08,"Prezado, Segue resposta formulada por Aline d...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
